In [1]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI

You can choose to use a local model through Ollama for example.

from langchain.llms import Ollama

ollama_llm = Ollama(model="openhermes") # 或是 mistral, codellama2, llama2

- 以下是 家裡桌機有下載的大模型, 20240210 推薦使用 mistral
- nexusraven 說明文件有強調它函數調用功能強化, 但目前我沒有測試過
- llava 是支援多模態的大模型, 但目前我沒有測試過
---
- codellama:latest                8fdf8f752f6e    3.8 GB  7 weeks ago
- llama2:latest                   78e26419b446    3.8 GB  2 weeks ago
- llama2-uncensored:latest        44040b922233    3.8 GB  10 days ago
- llava:latest                    cd3274b81a85    4.5 GB  11 days ago
- mistral:latest                  1ab49bc0b6a8    4.1 GB  7 weeks ago
- neural-chat:latest              f4c6a8e532e8    4.1 GB  7 weeks ago
- nexusraven:latest               483a8282af74    7.4 GB  2 weeks ago
- qwen:latest                     d53d04290064    2.3 GB  3 days ago
- starling-lm:latest              ff4752739ae4    4.1 GB  7 weeks ago

In [2]:
# crewai 使用 Ollama 的方法：windows 只需要打開 ubuntu 視窗即可，不需要執行 litellm；Mac 則甚至不需要啟動任何程式，只要有安裝好 Ollama 即可調用。
# 與 autogen 不同，autogen 要調用 Ollama 需要執行 litellm，多一道步驟，比較麻煩
gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.7),
ollama = Ollama(model="openhermes_assistant") 
gemini = ChatGoogleGenerativeAI(model = "gemini-pro", verbose = True, temperature = 0.6, google_api_key = os.environ["GOOGLE_API_KEY"])

In [3]:
# 簡易型的 tools 建立。就是從類別建立一個物件就可以使用。
# %pip install -U duckduckgo-search
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [4]:
# 建立 agents、tasks
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool]
   # You can pass an optional llm attribute specifying what mode you wanna use.
  # It can be a local model through Ollama / LM Studio or a remote
  # model like OpenAI, Mistral, Antrophic of others (https://python.langchain.com/docs/integrations/llms/)
  #
  # Examples:
  # llm=ollama_llm # was defined above in the file
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7)
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for
  your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  # verbose=True,
  # allow_delegation=True,
  # (optional) llm=ollama_llm
)


In [5]:
researcher.llm = gemini
writer.llm = gemini

In [6]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 4 paragraphs in traditional chinese 
   """,
  agent=writer
)

ValidationError: 1 validation error for Task
expected_output
  Field required [type=missing, input_value=<unprintable dict object>, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/missing

In [19]:
# Instantiate your crew with a sequential process 使用順序過程實例化您的船員
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=1, # You can set it to 1 or 2 to different logging levels
)

Overriding of current TracerProvider is not allowed


In [20]:
# Get your crew to work! kickoff(開球)開始工作~
result = crew.kickoff()

print("######################")
print(result)



> Entering new CrewAgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Thought: Do I need to use a tool? Yes
Action: duckduckgo_search
Action Input: latest advancements in AI in 2024Tool duckduckgo_search has been used.
 

In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI... 10 top AI and machine learning trends for 2024 10 top artificial intelligence certifications and courses for 2024 The future of AI: What to expect in the next 5 years The multimodal capabilities in OpenAI's GPT-4 model enable the software to respond to visual and audio input. Changes In Funding Gen AI And Other Technology In 2024 Feb 16, 2024,08:30am EST B2B Leaders, Here's How To Capitalize On Experience-Led Growth Feb 16, 2024,02:04am EST The Role Of... AI in 2024 — the biggest new products and advancements on the way Features By Ryan Morrison published 27 December 2023 While 2023 was the year of AI, 2024 will be the year we use it.... The most impor

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.



> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: duckduckgo_search
Action Input: What are the latest trends in generative AI that will impact the creative landscape?

d:\TOMO.Project\crewAI\venv\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


Tool duckduckgo_search has been used.
 

The biggest change has been the rise of generative AI, and particularly the use of transformers (a type of neural network) for everything from text and image generation to protein folding and... January 12, 2024 Generative AI trends started to take off in 2012, with various AI framework tools launched in many industries. Now, more than a decade later, AI has changed the workflow of businesses! We have AI Powered tools, services, and even analysis to understand customer perception. Published: 19 Apr 2023 Generative AI technology has percolated across multiple domains over the last few years. Much of this progress is due to advances in new large language models made possible by transformers. In addition to these insights, TechTarget's 2024 predictions for generative AI reveal several key trends. It is anticipated that there won't be a trough of disillusionment for Gen AI, as the technology continues to benefit enterprises. Multimodal AI is expecte

KeyboardInterrupt: 

d:\TOMO.Project\crewAI\venv\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")
